In [ ]:
!pip install deepface
!pip install dlib
!pip install labelme

In [ ]:
from deepface import DeepFace
import dlib
import cv2
import matplotlib.pyplot as plt
import os
import json
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/training_data'

In [ ]:
# get data
files = []

for root, dirs, file_names in os.walk(folder_path):
  for file_name in file_names:
    file_path = os.path.join(root, file_name)
    files.append(file_path)

print(f'Total number of files found: {len(files)}')
print(files)

In [ ]:
from PIL import Image, ImageDraw
import numpy as np

images = []
labels = []

for file in files:
  img = Image.open(file)
  img_array = np.array(img)
  images.append(img_array)
  label = file.split('/')[-1]
  labels.append(label)

In [ ]:
detector = dlib.get_frontal_face_detector()
for img_path in files[:5]:
  # load image
  image = cv2.imread(img_path)
  # convert to grayscale
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  # detect faces
  faces = detector(gray)
  # label the images
  file_name = img_path.split('/')[-1]
  label = file_name.split('.png')[-2]
  modified_label = f'{label}_detected.png'
  labels.append(modified_label)
  # loop through detected faces
  for face in faces:
    x, y, w, h = (face.left(), face.top(), face.width(), face.height())
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(image, modified_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

In [ ]:
# NEW CODE
import os
import cv2
from deepface import DeepFace

# Define the path to your dataset
dataset_path = '/content/drive/MyDrive/original_data'

# Get the list of known individuals
known_people = os.listdir(dataset_path)

# Create a dictionary to hold the known images and their labels
known_images = {}

# Load known images
for person in known_people:
    person_path = os.path.join(dataset_path, person)
    images = os.listdir(person_path)
    known_images[person] = [os.path.join(person_path, image) for image in images]


In [ ]:
import os
import pandas as pd
from deepface import DeepFace  # Ensure you have imported DeepFace

def recognize_faces(random_pictures_folder, dataset_path):
    results = []
    
    for image_file in os.listdir(random_pictures_folder):
        image_path = os.path.join(random_pictures_folder, image_file)
        try:
            # Analyze the image
            result = DeepFace.find(img_path=image_path, db_path=dataset_path)
            
            # Print the result to check its structure
            print(f"Result for {image_file}: {result}")

            # Check if result is not empty and is a list
            if isinstance(result, list) and len(result) > 0:  # Check if result is a list and not empty
                # Assuming the first element of the list is a DataFrame
                identity = result[0]['identity']  # Access the first match's identity
                
                # Check if identity is a Series
                if isinstance(identity, pd.Series):
                    recognized_person = identity.iloc[0].split('/')[-2]  # Extract the label from the Series
                else:
                    recognized_person = identity.split('/')[-2]  # Regular string case

                results.append((image_file, recognized_person))
            else:
                results.append((image_file, "Unknown"))

        except Exception as e:
            results.append((image_file, f"Error: {str(e)}"))

    return results


# Update these paths to your actual dataset and random images folder
random_pictures_folder = '/content/drive/MyDrive/random_data'  # Replace with your random images folder path
dataset_path = '/content/drive/MyDrive/original_data'  # Set your dataset path

recognition_results = recognize_faces(random_pictures_folder, dataset_path)

# Print results
for image, recognized_person in recognition_results:
    print(f"{image}: {recognized_person}")


In [ ]:
def calculate_accuracy(predictions, ground_truth):
    correct = sum(1 for p, gt in zip(predictions, ground_truth) if p == gt)
    accuracy = correct / len(ground_truth) * 100
    return accuracy

# Assume ground_truth is a list of labels corresponding to your random pictures
ground_truth = ['african', 'african', 'african', 'caucasian', 'caucasian', 'indian', 'indian']  # Fill this with the true labels for your random images

# Extract predicted labels from recognition_results
predicted_labels = [result[1] for result in recognition_results]

# Calculate accuracy
accuracy = calculate_accuracy(predicted_labels, ground_truth)
print(f"Accuracy: {accuracy:.2f}%")


In [ ]:
# face matching and finding face in database
# we can do custom face detection - by setting detector_backend
result = DeepFace.verify(img1_path="/content/data/Caucasian_36_37.png", img2_path="/content/data/Caucasian_36_73.png")
print(json.dumps(result, indent = 2))

In [ ]:
# finding face in db
dfs = DeepFace.find(img_path="/content/data/Asian_58_1.png", db_path="/content/data")
print(dfs)